## Music Genre Analysis

### Imports

In [ ]:
# Library imports

import pandas as pd

pd.set_option("display.max_columns", None)

### Importing the dataset and basic EDA

In [ ]:
# Dataset import

df = pd.read_csv("./Data/features_30_sec.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

#### General data explanation

Each of the musical characteristics is split into a pair of '_mean' and '_var' labeled columns:

- **mean** - represents the mean of characteristic value
- **var** - represents the variance of characteristic value

Characteristics of the dataset samples:

- **chroma_stft** - (Chroma Short-Time Fourier Transformation) - represents energy distribution of pitches in audio signals
- **rms** - (Root Mean Square) - represents the amplitude of the audio signal
- **spectral_centroid** - indicates where the center of mass of the spectrum is located and gives an idea about the sound brightness
- **spectral_bandwidth** - describes the width of the spectral band around the spectral centroid
- **rolloff** - (Spectral roll-off) - represents the frequency below which a specified percentage of the total spectral energy is contained
- **zero_crossing_rate** - Indicated the rate at which the signal changes its sign, related to sound noisiness or percussiveness
- **harmony** - represents the overall tonal quality and harmonic content of the sound
- **perceptr** - (Perceptual spread) - represents the perceived spread of the sound frequency
- **tempo** - represents the pace or speed of the music sample, measured in beats per minute (BPM)
- **mfcc** - (Mel-Frequency Cepstral Coefficients) - coefficient representation of the short-term power spectrum of the sound signal

In [ ]:
df.describe().transpose()

In [ ]:
# Check for missing values

df.isna().sum()

In [ ]:
df['label'].value_counts().reset_index()

In [ ]:
from matplotlib import pyplot as plt

# Drop str type columns from the dataset

df_trimmed = df.drop(columns=["filename", "label"], axis=1)
columns_to_plot = df_trimmed.columns

# Set up subplots

fig, axes = plt.subplots(nrows=len(columns_to_plot), ncols=1, figsize=(12, 60))

# Plot a histogram for each column

for i, column in enumerate(columns_to_plot):
    axes[i].hist(df_trimmed[column], color='blue', alpha=0.7)
    axes[i].set_title(column)

plt.tight_layout()
plt.show()


In [ ]:
import re

# Remove all of the mfcc*_vars and mfcc*_means except for the initial ones

columns = df_trimmed.columns

columns_to_drop = []
first_occurances = {}

for column in columns:
    if column.startswith("mfcc") and ("_mean" in column or "_var" in column):
        match = re.search(r"mfcc(\d+)_", column)
        if match and int(match.group(1)) > 1:
            columns_to_drop.append(column)

df_trimmed.drop(columns=columns_to_drop, inplace=True)

print(df_trimmed.columns)
print(f'Columns: {len(df_trimmed.columns)}')

In [ ]:
# Draw the correlation matrix

import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = df_trimmed.corr()

plt.figure(figsize=(16, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation matrix")
plt.show()

### Visualizing the data

In [ ]:
# Get every unique genre in the dataset

genres = df['label'].unique()
print(genres)

In [ ]:
# Create a dictionary with file names for each music genre

import os

audio_dir_contents = {}
for genre in genres:
    audio_files = [
        file
        for file in os.listdir(f"./Data/genres_original/{genre}")
        if file.endswith(".wav")
    ]
    audio_dir_contents[genre] = audio_files

print(audio_dir_contents)

In [ ]:
# Select a random file from each music genre

import random

random_audio_selection = {}
for genre, files in audio_dir_contents.items():
    random_audio = random.choice(files)
    random_audio_selection[genre] = random_audio

print(random_audio_selection)

In [ ]:
# Load the audio with Librosa

import librosa

audios = {}
sample_rates = {}
for audio in random_audio_selection.keys():
    audio_path = f'./Data/genres_original/{audio}/{random_audio_selection[audio]}'
    data, sr = librosa.load(audio_path, sr=45600)
    audios[audio] = data
    sample_rates[f'{audio}:sr'] = sr

In [ ]:
# Add a color dictionary for audio plots

audio_colors = {
    'blues': 'blue',
    'classical': 'purple',
    'country': 'brown',
    'disco': 'gray',
    'hiphop': 'red',
    'jazz': 'violet',
    'metal': 'black',
    'pop': 'pink',
    'reggae': 'green',
    'rock': 'orange'
}

#### Audio plot descriptions

- **Waveplot** - represents the change of air pressure at the current moment, ie. loudness of the audio sample at a given moment
- **STFT plot** - represents frequency components and their change over time
- **Spectrogram plot** - represents how the frequency content of a signal changes over time
- **Spectral roll-off plot** - represents the distribution of energy across the frequency spectrum
- **MFCC** - represents the frequency composition and the dynamics of the audio samples

In [ ]:
# Waveplots

for audio in audios.keys():
    plt.figure(figsize=(12, 4))
    plt.title(f'Waveplot for {audio} music genre', size=15)
    librosa.display.waveshow(audios[audio], color=audio_colors[audio])
    plt.show()

In [ ]:
# STFT plots

for audio in zip(audios.keys(), sample_rates.keys()):
    stft = librosa.stft(audios[audio[0]])
    plt.figure(figsize=(12, 4))
    plt.title(f'Short Time Fourier Transformation plot for {audio[0]} music genre', size=15)
    librosa.display.specshow(stft, sr=sample_rates[audio[1]], x_axis='time', y_axis='hz')
    plt.colorbar()

In [ ]:
# Spectrogram plots

for audio in zip(audios.keys(), sample_rates.keys()):
    stft = librosa.stft(audios[audio[0]])
    stft_db = librosa.amplitude_to_db(abs(stft))
    plt.figure(figsize=(12, 4))
    plt.title(f'Spectrogram plot for {audio[0]} music genre', size=15)
    librosa.display.specshow(stft_db, sr=sample_rates[audio[1]], x_axis='time', y_axis='hz')
    plt.colorbar()

In [ ]:
# Spectral roll-offs plots

for audio in zip(audios.keys(), sample_rates.keys()):
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audios[audio[0]], sr=sample_rates[audio[1]])[0]
    plt.figure(figsize=(12, 4))
    plt.title(f'Spectral Rolloff plot for {audio[0]} music genre', size=15)
    plt.plot(spectral_rolloff, color=audio_colors[audio[0]])
    plt.xlabel('Frame')
    plt.ylabel('Spectral Rolloff')
    plt.show()

In [ ]:
# MFCC plots

for audio in zip(audios.keys(), sample_rates.keys()):
    mfcc = librosa.feature.mfcc(y=audios[audio[0]], sr=sample_rates[audio[1]])
    plt.figure(figsize=(12, 4))
    plt.title(f'MFCC plot for {audio[0]} music genre', size=15)
    librosa.display.specshow(mfcc, sr=sample_rates[audio[1]], x_axis='time')
    plt.show()